In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 chat - Multi-turn

* [Google Gen AI SDKs](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)
  * The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both.
  * This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [python-genai](https://googleapis.github.io/python-genai/)

In [2]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [4]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-exp" # @param {type:"string"}

### Import libraries

In [5]:
import base64

from google import genai
from google.genai.chats import Chats

from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)


### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [6]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [8]:
def generate_content_config():
  """
  Method to configure generation config.

  """

  system_instruction = """
      당신은 최신 정보를 포함한 다양한 사용자의 질문에 답을 해주는 AI Assistant 입니다.
      답변은 간략하게 10줄 이내로 설명해 주세요.
  """


  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                      SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]

  google_search_tool = Tool(google_search=GoogleSearch())

  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    top_k=20,
    candidate_count=1,
    seed = 5,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
    tools = [google_search_tool],
    system_instruction=system_instruction,

  )

  return generate_content_config


def chat_gemini_2_0(chat:Chats, query:str) -> GenerateContentResponse:

  # https://googleapis.github.io/python-genai/genai.html#genai.chats.Chats.create
  response = chat.send_message(query)
  return response


### Execution

In [ ]:
gen_config = generate_content_config()
# https://googleapis.github.io/python-genai/genai.html#genai.chats.Chats.create
chat = client.chats.create(model=MODEL_NAME, config=gen_config)

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""

  답변은 최대한 간결하게 답해주세요.

  질문 : {query}
  """

  response = chat_gemini_2_0(chat, prompt)
  display(Markdown(f"AI Agent : {response.text}"))
  print(f"------------------------------------ ")


사용자: 안녕하세요.


AI Agent : 네, 안녕하세요. 무엇을 도와드릴까요?


------------------------------------ 
사용자: 한국의 최근 정치 상황에 대해서 알려주세요.


AI Agent : 최근 한국 정치 상황은 매우 불안정하며 복잡합니다. 윤석열 대통령이 계엄령을 선포했다가 철회한 후, 국회에서 탄핵소추안이 가결되어 직무가 정지되었습니다. 현재 한덕수 국무총리가 대통령 권한대행을 맡고 있으며, 헌법재판소에서 탄핵 심판이 진행될 예정입니다.

**주요 사건:**

*   **계엄령 선포 및 철회:** 윤석열 대통령이 국회와 야당을 비판하며 계엄령을 선포했으나 즉시 철회되었습니다.
*   **탄핵소추안 가결:** 국회에서 윤석열 대통령에 대한 탄핵소추안이 가결되어 대통령 직무가 정지되었습니다.
*   **헌법재판소 심판:** 헌법재판소에서 탄핵 심판이 진행될 예정이며, 최대 6개월이 소요될 수 있습니다.
*   **정치적 양극화:** 여당과 야당 간의 갈등이 심화되어 정치적 불안이 가중되고 있습니다.
*   **경제적 영향:** 정치 불안정은 경제에도 부정적인 영향을 미치고 있으며, 투자 심리 위축 및 소비 부진을 유발할 수 있다는 우려가 있습니다.

**향후 전망:**

*   헌법재판소의 탄핵 심판 결과에 따라 대통령의 거취가 결정될 것입니다.
*   정치적 불확실성이 지속될 가능성이 높으며, 이에 따른 사회적 혼란과 경제적 영향이 우려됩니다.
*   국내외적으로 한국 정치 상황에 대한 우려의 목소리가 나오고 있습니다.

요약하자면, 현재 한국은 대통령 탄핵과 정치적 양극화로 인해 매우 혼란스러운 상황이며, 향후 정치 및 경제적 불확실성이 높은 시기입니다.


------------------------------------ 
사용자: 부동산에 어떤 영향을 줄까요 ?


AI Agent : 한국의 최근 정치 상황은 부동산 시장에 다음과 같은 영향을 미칠 수 있습니다:

**단기적 영향:**

*   **거래량 감소:** 정치적 불확실성으로 인해 주택 구매 심리가 위축되어 거래량이 감소할 수 있습니다.
*   **가격 하락 가능성:** 거래 감소와 함께 집값 하락으로 이어질 가능성이 있으며, 특히 투자 목적의 부동산 수요가 줄어들 수 있습니다.
*   **관망세 확산:** 시장 참여자들이 상황을 지켜보며 관망하는 태도를 취할 수 있습니다.

**장기적 영향:**

*   **정책 변화 가능성:** 탄핵 결과에 따라 차기 정부의 부동산 정책 방향이 달라질 수 있으며, 이는 시장에 큰 영향을 미칠 수 있습니다.
*   **건설업계 위축:** 공공기관 발주 사업이 지연되거나 축소될 수 있으며, 이는 건설업계에 부정적인 영향을 줄 수 있습니다.
*   **금리 변동:** 정치적 불안정은 금리 변동으로 이어질 수 있으며, 이는 대출 금리에 영향을 주어 주택 구매력에 영향을 미칠 수 있습니다.

**과거 사례:**

*   **박근혜 전 대통령 탄핵 당시:** 거래량과 가격이 단기적으로 감소했으나, 정치적 불확실성이 해소된 후 시장이 회복되었습니다.
*   **노무현 전 대통령 탄핵 당시:** 탄핵보다는 대외 변수에 따라 부동산 시장이 움직였습니다.

**전문가 의견:**

*   탄핵 자체보다는 대출 규제, 금리, 경제 상황 등 다른 요인들이 부동산 시장에 더 큰 영향을 미칠 수 있습니다.
*   단기적인 시장 변동에 크게 반응하기보다는 장기적인 관점에서 시장을 지켜보는 것이 좋습니다.

**결론:**

정치적 불안정은 부동산 시장에 부정적인 영향을 미칠 수 있지만, 단기적인 영향에 일희일비하기보다는 장기적인 관점에서 시장을 주시하는 것이 중요합니다. 또한, 금리, 대출 정책, 경제 상황 등 다른 변수들도 함께 고려해야 합니다.


------------------------------------ 
사용자: 최근 12월에 논의된 AI 트렌드에 대해서 설명해주세요.


AI Agent : 2024년 12월, AI 분야에서는 다음과 같은 주요 트렌드가 논의되었습니다.

1.  **AI 민주화:** AI 기술이 전문가뿐 아니라 일반 사용자도 쉽게 접근하고 활용할 수 있도록 보급되고 있습니다. 이는 코딩 없이 AI 도구를 사용할 수 있는 플랫폼과 서비스의 증가로 이어지고 있습니다.

2.  **검색 엔진에 AI 통합:** 구글과 같은 주요 검색 엔진들이 기존의 키워드 검색 방식을 AI 기반의 대화형 검색으로 전환하고 있습니다. 이를 통해 사용자들은 보다 정확하고 상세한 정보를 빠르게 얻을 수 있습니다.

3.  **다중 모드 AI 시스템:** 텍스트, 이미지, 음성 등 다양한 형태의 데이터를 처리할 수 있는 AI 모델이 발전하고 있습니다. 이는 AI가 더욱 다양한 분야에서 활용될 수 있도록 합니다.

4.  **인간과 로봇의 협업:** 로봇이 인간과 함께 작업하는 환경이 늘어나고 있으며, AI는 로봇의 센서, 학습 능력, 의사 결정 능력을 향상시키는 데 기여하고 있습니다.

5.  **AI 시장 경쟁 심화:** AI 기술의 발전에 따라 시장 경쟁이 더욱 치열해지고 있습니다. 특히 클라우드 서비스와 소프트웨어 시장에서 AI 기업들의 경쟁이 두드러지게 나타나고 있습니다.

6.  **지적 재산권 문제:** AI가 저작권이 있는 콘텐츠를 학습하는 것에 대한 우려가 커지고 있으며, 이에 대한 해결책을 찾기 위한 논의가 활발하게 이루어지고 있습니다.

이러한 트렌드를 통해 AI는 우리의 일상생활과 다양한 산업 분야에서 더욱 중요한 역할을 하게 될 것으로 예상됩니다.


------------------------------------ 
